In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_json('./data/intervals_challenge.json', lines=True)

In [3]:
data.head()

,duration,distance,started_at,trip_id,vehicle_id,city_id,type
0,857,5384,2016-10-03 13:00:00.286999941,c00cee6963e0dc66e50e271239426914,52d38cf1a3240d5cbdcf730f2d9a47d6,pentos,driving_to_destination
1,245,1248,2016-10-03 13:00:00.852999926,427425e1f4318ca2461168bdd6e4fcbd,8336b28f24c3e7a1e3d582073b164895,volantis,going_to_pickup
2,1249,5847,2016-10-03 13:00:01.670000076,757867f6d7c00ef92a65bfaa3895943f,8885c59374cc539163e83f01ed59fd16,pentos,driving_to_destination
3,471,2585,2016-10-03 13:00:01.841000080,d09d1301d361f7359d0d936557d10f89,81b63920454f70b6755a494e3b28b3a7,bravos,going_to_pickup
4,182,743,2016-10-03 13:00:01.970000029,00f20a701f0ec2519353ef3ffaf75068,b73030977cbad61c9db55418909864fa,pentos,going_to_pickup


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165170 entries, 0 to 165169
Data columns (total 7 columns):
duration      165170 non-null object
distance      165170 non-null object
started_at    165170 non-null datetime64[ns]
trip_id       165170 non-null object
vehicle_id    165170 non-null object
city_id       165170 non-null object
type          165170 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 8.8+ MB


### The experiment wants to know how to assign rides to the closest vehicle. Thus, we only want to use the optimization when 'going_to_pickup'. That's why we create a new dataframe grouping by this information only

In [5]:
pickups = data[data.type == 'going_to_pickup']

### As it can be seen below now we only have the type of trips we are interested in

In [6]:
pickups.head()

,duration,distance,started_at,trip_id,vehicle_id,city_id,type
1,245,1248,2016-10-03 13:00:00.852999926,427425e1f4318ca2461168bdd6e4fcbd,8336b28f24c3e7a1e3d582073b164895,volantis,going_to_pickup
3,471,2585,2016-10-03 13:00:01.841000080,d09d1301d361f7359d0d936557d10f89,81b63920454f70b6755a494e3b28b3a7,bravos,going_to_pickup
4,182,743,2016-10-03 13:00:01.970000029,00f20a701f0ec2519353ef3ffaf75068,b73030977cbad61c9db55418909864fa,pentos,going_to_pickup
5,599,1351,2016-10-03 13:00:02.154000044,158e7bc8d42e1d8c94767b00c8f89568,126e868fb282852c2fa95d88878686bf,volantis,going_to_pickup
9,1525,2674,2016-10-03 13:00:05.637000084,d3e6e8fb50c02d66feca2c60830c4fcc,b0906e917dc5cc0bcba190fd80079a74,bravos,going_to_pickup


### We have quenched the data to 58.510 rows showing the information we want to study

In [7]:
pickups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58510 entries, 1 to 165169
Data columns (total 7 columns):
duration      58510 non-null object
distance      58510 non-null object
started_at    58510 non-null datetime64[ns]
trip_id       58510 non-null object
vehicle_id    58510 non-null object
city_id       58510 non-null object
type          58510 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 3.6+ MB


### We check if there are any irregular values or nulls. We sort them, to see if among the numbers, are there any non-numeric

In [8]:
pickups['duration'].sort_values

<bound method Series.sort_values of 1          245
3          471
4          182
5          599
9         1525
          ... 
165153      NA
165156      NA
165158      NA
165159      NA
165169      NA
Name: duration, Length: 58510, dtype: object>

### Drop the nulls

In [9]:
pickups = pickups[pickups.duration != 'NA']

### We turn duration and distance into numerics 

In [10]:
pickups[['duration', 'distance']] = pickups[['duration', 'distance']].apply(pd.to_numeric)

In [11]:
pickups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58211 entries, 1 to 165111
Data columns (total 7 columns):
duration      58211 non-null int64
distance      58211 non-null int64
started_at    58211 non-null datetime64[ns]
trip_id       58211 non-null object
vehicle_id    58211 non-null object
city_id       58211 non-null object
type          58211 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 3.6+ MB


### We generate a function to discriminate the kind of distance used to assign the call to the vehicle

In [12]:
def linear_road(row):
    if re.match(r'^[0-8]', row['trip_id']):
        return 'road'
    else:
        return 'linear'

### Creation of a new column with the result of the function

In [13]:
pickups['trip_type'] = pickups.apply(linear_road, axis=1)

In [14]:
pickups.head()

,duration,distance,started_at,trip_id,vehicle_id,city_id,type,trip_type
1,245,1248,2016-10-03 13:00:00.852999926,427425e1f4318ca2461168bdd6e4fcbd,8336b28f24c3e7a1e3d582073b164895,volantis,going_to_pickup,road
3,471,2585,2016-10-03 13:00:01.841000080,d09d1301d361f7359d0d936557d10f89,81b63920454f70b6755a494e3b28b3a7,bravos,going_to_pickup,linear
4,182,743,2016-10-03 13:00:01.970000029,00f20a701f0ec2519353ef3ffaf75068,b73030977cbad61c9db55418909864fa,pentos,going_to_pickup,road
5,599,1351,2016-10-03 13:00:02.154000044,158e7bc8d42e1d8c94767b00c8f89568,126e868fb282852c2fa95d88878686bf,volantis,going_to_pickup,road
9,1525,2674,2016-10-03 13:00:05.637000084,d3e6e8fb50c02d66feca2c60830c4fcc,b0906e917dc5cc0bcba190fd80079a74,bravos,going_to_pickup,linear


### We drop the rides called from less than 50 metres from the spot they were called, because most likely they are meaningless

In [15]:
pickups = pickups[pickups['distance'] > 50]

In [16]:
pickups.distance.sort_values()

144276         51
117208         51
119783         51
34596          51
29252          51
           ...   
42689      890705
89181      892112
33646     1137902
33526     1137915
37214     1218089
Name: distance, Length: 54583, dtype: int64

### We group by the type of trip we program, to compare the results

In [17]:
pickups.groupby('trip_type').mean()

,duration,distance
trip_type,,
linear,315.067671,1034.966778
road,315.702203,1059.789300


### Apparently there is no difference whatsoever depending on which model assigns the car to pick up the customer. To understand if this is so, we groupby cities as well

In [19]:
pickups.groupby(['city_id', 'trip_type']).mean()

duration     distance
city_id  trip_type                         
bravos   linear     591.711879  3228.833193
         road       616.431222  3265.321177
pentos   linear     266.093769   746.598132
         road       264.985662   774.577343
volantis linear     343.224252   931.508131
         road       335.375653   930.273488

## Conclusion

### 1.1. Should the company move towards road distance? 

They shouldn't in overall, though the specificity of the city of Volantis might make it interesting to apply it there

### 1.2. What's the max price it would make sense to pay per query?

They should pay a price per query smaller than the difference of revenue they obtain without that system.

### 2. How would you improve the experimental design? Would you collect any additional data?

To make sure the experiment is worth being applied we would need the price of a query and the price the company earns per ride. Based on these columns, we would calculate the difference in the revenue for the company.